In [2]:
#!/usr/bin/env python
# coding: utf-8

import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
import openpyxl
from openpyxl import load_workbook
from keras import backend as K
import gc
import tensorflow as tf


file_path_spy = '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/spy.csv'
file_path_cash = '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/cash.csv'

# 读取数据
data_spy = pd.read_csv(file_path_spy)
data_cash = pd.read_csv(file_path_cash)

# 确保日期列格式为datetime
data_spy['date'] = pd.to_datetime(data_spy['date'])
data_cash['date'] = pd.to_datetime(data_cash['date'])

# 创建 return_label 列
data_spy['return_label'] = data_spy['return'].apply(lambda x: '+' if x > 0 else '-')
data_cash['return_label'] = data_cash['return'].apply(lambda x: '+' if x > 0 else '-')

# 构建特征
def create_label_sequence(data):
    sequences = []
    for i in range(len(data)):
        if i >= 9:  # 确保有足够的数据来创建10天的序列
            sequence = ''.join(data['return_label'][i-10:i])
            sequences.append(sequence)
        else:
            sequences.append(None)  # 对于序列开始的部分，填充None
    return sequences

data_spy['feature'] = create_label_sequence(data_spy)
data_cash['feature'] = create_label_sequence(data_cash)

# 合并特征和目标变量，只保留spy的特征
combined_data = pd.merge(data_spy, data_cash, on='date')
combined_data['combined_feature'] = combined_data['feature_x']  # 只使用spy的特征
combined_data['target'] = combined_data.apply(lambda row: 'spy' if row['return_x'] > row['return_y'] else 'cash', axis=1)

# 选择相关列
columns_to_display = ['date', 'target', 'combined_feature', 'return_label_x', 'return_label_y', 'return_x', 'return_y']
display_data = combined_data[columns_to_display]

# 保存结果到Excel文件
output_excel_path = 'combined_data.xlsx'
display_data.to_excel(output_excel_path, index=False)

# 保存结果到CSV文件
output_csv_path = 'combined_data.csv'
display_data.to_csv(output_csv_path, index=False)

print("Data has been saved to", output_excel_path, "and", output_csv_path)


Data has been saved to combined_data.xlsx and combined_data.csv
